Importiamo le librerie necessarie per lo scraping e la creazione dei Dataset di test in files con estensione ".csv".

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, ElementNotInteractableException
from pandas import DataFrame
from os import chdir
from sys import path

Per comodità eseguiamo lo script nella stessa cartella in cui è presente il file.

In [ ]:
chdir(path[0])

Creiamo il dizionario che sarà utile per relazionare il nome del film con il suo relativo link all'IMDb. Abbiamo ordinato le recensioni dalla più recente, evitando di posizionare le più votate o quelle con i voti più alti nelle prime posizioni. La cosa è comunque non particolarmente rilevante dal momento che andremo ad eseguire lo scraping di tutte le recensioni per tutti i film, senza lasciare la possibilità di aver analizzato un campione non rappresentativo dell'intera popolazione di recensioni.

In [ ]:
films = {
    'Iron Man': 'https://www.imdb.com/title/tt0371746/reviews?sort=submissionDate&dir=desc&ratingFilter=0',
    'L\'incredibile Hulk': 'https://www.imdb.com/title/tt0800080/reviews?sort=submissionDate&dir=desc&ratingFilter=0',
    'Iron Man 2': 'https://www.imdb.com/title/tt1228705/reviews?sort=submissionDate&dir=desc&ratingFilter=0',
    'Thor': 'https://www.imdb.com/title/tt0800369/reviews?sort=submissionDate&dir=desc&ratingFilter=0',
    'Captain America - Il primo vendicatore': 'https://www.imdb.com/title/tt0458339/reviews?sort=submissionDate&dir=desc&ratingFilter=0',
    'The Avengers': 'https://www.imdb.com/title/tt0848228/reviews?sort=submissionDate&dir=desc&ratingFilter=0',
    'Iron Man 3': 'https://www.imdb.com/title/tt1300854/reviews?sort=submissionDate&dir=desc&ratingFilter=0',
    'Thor - The Dark World': 'https://www.imdb.com/title/tt1981115/reviews?sort=submissionDate&dir=desc&ratingFilter=0',
    'Captain America - The Winter Soldier': 'https://www.imdb.com/title/tt1843866/reviews?sort=submissionDate&dir=desc&ratingFilter=0',
    'Guardiani della Galassia': 'https://www.imdb.com/title/tt2015381/reviews?sort=submissionDate&dir=desc&ratingFilter=0',
    'Avengers - Age of Ultron': 'https://www.imdb.com/title/tt2395427/reviews?sort=submissionDate&dir=desc&ratingFilter=0',
    'Ant-Man': 'https://www.imdb.com/title/tt0478970/reviews?sort=submissionDate&dir=desc&ratingFilter=0',
    'Captain America - Civil War': 'https://www.imdb.com/title/tt3498820/reviews?sort=submissionDate&dir=desc&ratingFilter=0',
    'Doctor Strange': 'https://www.imdb.com/title/tt1211837/reviews?sort=submissionDate&dir=desc&ratingFilter=0',
    'Guardiani della Galassia Vol. 2': 'https://www.imdb.com/title/tt3896198/reviews?sort=submissionDate&dir=desc&ratingFilter=0',
    'Spider-Man - Homecoming': 'https://www.imdb.com/title/tt2250912/reviews?sort=submissionDate&dir=desc&ratingFilter=0',
    'Thor - Ragnarok': 'https://www.imdb.com/title/tt3501632/reviews?sort=submissionDate&dir=desc&ratingFilter=0',
    'Black Panther': 'https://www.imdb.com/title/tt3501632/reviews?sort=submissionDate&dir=desc&ratingFilter=0',
    'Avengers - Infinity War': 'https://www.imdb.com/title/tt4154756/reviews?sort=submissionDate&dir=desc&ratingFilter=0',
    'Ant-Man and the Wasp': 'https://www.imdb.com/title/tt5095030/reviews?sort=submissionDate&dir=desc&ratingFilter=0',
    'Captain Marvel': 'https://www.imdb.com/title/tt4154664/reviews?sort=submissionDate&dir=desc&ratingFilter=0',
    'Avengers - Endgame': 'https://www.imdb.com/title/tt4154796/reviews?sort=submissionDate&dir=desc&ratingFilter=0',
    'Spider-Man - Far from Home': 'https://www.imdb.com/title/tt6320628/reviews?sort=submissionDate&dir=desc&ratingFilter=0',
    'Black Widow': 'https://www.imdb.com/title/tt3480822/reviews?sort=submissionDate&dir=desc&ratingFilter=0',
    'Shang-Chi e la leggenda dei Dieci Anelli': 'https://www.imdb.com/title/tt9376612/reviews?sort=submissionDate&dir=desc&ratingFilter=0',
    'Eternals': 'https://www.imdb.com/title/tt9032400/reviews?sort=submissionDate&dir=desc&ratingFilter=0',
    'Spider-Man - No Way Home': 'https://www.imdb.com/title/tt10872600/reviews?sort=submissionDate&dir=desc&ratingFilter=0'

}

Istanziamo una variabile con il percorso del **WebDriver**, nel nostro caso abbiamo utilizzato quello relativo al **browser Google Chrome**.

In [ ]:
path = 'chromedriver.exe'

Inizializziamo l'oggetto "pilot" di tipo WebDriver utilizzando alcune opzioni utili dalla [documentazione](https://www.selenium.dev/selenium/docs/api/py/index.html). È possibile eventualmente aggiungere altre opzioni come "start-maximized", che esegue la finestra a tutto schermo oppure "--headless", per non visualizzare a schermo il browser ma lasciarlo in background. Abbiamo lasciato la riga di codice con queste opzioni come commento per lasciare la possibilità di decidere come procedere volta per volta

In [ ]:
s=Service(path)
options = webdriver.ChromeOptions()
#options.add_argument("--headless", "start-maximized")
options.add_experimental_option('excludeSwitches', ['enable-logging'])
pilot = webdriver.Chrome(service=s, options=options)

Con un *ciclo for* eseguiamo lo scraping per tutti i film nel dizionario, ottenendo **chiavi** e **valori** con il metodo *.items()*

In [ ]:
for key, value in films.items():

Il primo passo da compiere all'interno del *ciclo for* è inizializzare il browser sulla pagina delle recensioni, di conseguenza passiamo al **WebDriver** il link del film, che cambierà in base al numero di iterazioni del *ciclo for*.

In [ ]:
    pilot.get(value)

La pagina web relativa alle recensioni dei film, mostra di default solo le prime 25 recensioni, e solo premendo un *bottone* in fondo alla pagina che riporta la dicitura *"Load more..."* è possibile caricarne altre 25. Il codice HTML risulta così incompleto e poco utile ai fini dello scraping. La soluzione è premere il *bottone* in questione ogni volta finchè non verranno visualizzate tutte le recensioni relative al film, così che il codice HTML venga caricato mano mano e reso completo per poterlo analizzare.
Implementiamo quindi un *ciclo while* senza condizione di uscita.

In [ ]:
    while True:

All'interno del *while* viene eseguita l'istruzione per cliccare sul *bottone* ripetutamente. Gestiamo le possibili eccezioni **NoSuchElementException**, **ElementClickInterceptedException** e **ElementNotInteractableException**:
1. **NoSuchElementException** si verifica se non è stato trovato il *bottone* che rispetta le condizioni di ricerca (potrebbe succedere ad esempio nel caso in cui le recensioni di un particolare film sono di meno di 25 e il bottone ovviamente non è presente). 
2. **ElementClickInterceptedException** si verifica quando un altro elemento HTML intercetta lo script (generalmente questo potrebbe succedere quando il padre di un elemento intercetta il click destinato a uno dei figli).
3. **ElementNotInteractableException** si verifica quando il click è destinato a un elemento non cliccabile, che non esegue alcun reindirizzamento o alcuna funzione.

Dai nostri test abbiamo rilevato che, quando si verifica qualsiasi tra le tre, è perchè o il *bottone* non è presente o con il *ciclo while* si sono già caricate tutte le recensioni possibili. Di conseguenza non gestiamo l'eccezione in maniera particolare, ma inseriamo semplicemente l'istruzione *break* per fermare il ciclo e proseguire con lo scraping.

In [ ]:
        try:
            pilot.find_element(By.XPATH, '//div[@class="load-more-data"]').click()
        except NoSuchElementException:
            break
        except (ElementClickInterceptedException, ElementNotInteractableException):
            break

Durante lo scraping abbiamo incontrato un ulteriore difficoltà. IMDb dà la possibilità a chi scrive le recensioni di abilitare **un'opzione di spoiler** che nasconda il contenuto testuale della recensione, così che un qualsiasi utente che si trova nella pagina può scegliere liberamente se visualizzare un particolare contenuto o meno, senza appunto andare incontro a un brutto spoiler. Non tutte le recensioni sono nascoste dal momento che alcuni utenti si limitano a recensire il film senza fare alcun riferimento alla trama. Avremmo potuto analizzare solo queste, ma per completezza abbiamo voluto includerle tutte e per questo motivo si è reso necessario questo ulteriore passaggio.
Istanziamo una lista *warning_buttons* che contiene tutti gli elementi **WebElement** associati ai *bottoni* che permettono di visualizzare la recensione con spoiler.

In [ ]:
    warning_buttons = pilot.find_elements(By.XPATH, './/div[@class="expander-icon-wrapper spoiler-warning__control"]')

Implementiamo un *ciclo for* che ci permette di eseguire operazioni per ognuno dei *bottoni* presenti nella lista *warning_buttons*

In [ ]:
    for button in warning_buttons:

Proviamo a fare click sul *bottone di spoiler*

In [ ]:
        try:
            button.click()

Potrebbero come al solito essere generate eccezioni. Mentre nel caso precedente abbiamo semplicemente potuto fermare il ciclo, qui non è possibile perchè, se il click venisse intercettato bisogna trovare un modo per reindirizzarlo al destinatario originale. Abbiamo pensato di compiere quest'operazione passando all'oggetto **WebDriver** una stringa da eseguire come *script JavaScript*.

In [ ]:
        except (ElementClickInterceptedException, ElementNotInteractableException):
            pilot.execute_script("arguments[0].click();", button)

Arrivati a questo punto il codice HTML è completo e possiamo procedere con il vero e proprio scraping.
Individuiamo gli elementi **WebElement** che corrispondono ai box che contengono le intere recensioni.

In [ ]:
    reviews = pilot.find_elements(By.XPATH, './/div[@class="lister-item-content"]')

Il box, oltre a contenere il testo recensione, possiede altri elementi come ad esempio la data in cui è stata scritta, il nickname dell'autore, il voto attribuito al film, il titolo ecc.
Istanziamo due liste in cui andremo ad aggiungere gli elementi che ci interessano di ogni recensione, ovvero il voto e il testo.

In [ ]:
    ratings = []
    texts = []

Con un altro *ciclo for* scorriamo la *lista* di oggetti **WebElement**

In [ ]:
    for review in reviews:

Partiamo dal voto. Lo catturiamo e lo aggiungiamo alla lista precedentemente istanziata.
Abbiamo trovato, durante i nostri test, alcune recensioni prive di valutazione in decimi. Per come abbiamo poi implementato il progetto, abbiamo bisogno di recensioni che rispettino tutte lo stesso standard, e di conseguenza abbiamo escluso le suddette che sono davvero in numero esiguo (poche decine su migliaia).

In [ ]:
        try:
            rating = review.find_element(By.XPATH, './/div[@class="ipl-ratings-bar"]').text.replace('\n', '')
            ratings.append(rating.split('/')[0])
        except NoSuchElementException:
            continue

Lo stesso procedimento vale per il titolo,

In [ ]:
        title = review.find_element(By.XPATH, './/a[@class="title"]').text.replace('\n', '')

E per il contenuto testuale.

In [ ]:
        body = review.find_element(By.XPATH, './/div[contains(@class, "text show-more__control")]').text.replace('\n', '')
        texts.append(f'{title} {body}')

A questo punto è finito il ciclo interno, le liste sono piene, e quindi organizziamo un *DataFrame* sfruttando la *libreria Pandas*.

In [ ]:
    df = DataFrame({
        'Rating' : ratings,
        'Reviews' : texts
    })

Per ogni DataFrame, di ogni film, creiamo un file con estensione ".csv" con il nome del film che abbiamo utilizzato nel dizionario, così da poter organizzare i file per nome.
Li posizioniamo nella cartella Reviews appositamente creata.

In [ ]:
    df.to_csv(f'.\Reviews\{key}.csv')

Le recensioni sono davvero tante, e aumentano ogni giorno nonostante alcuni dei film analizzati sono vecchi di anni.</br>